In [3]:
# 源代码安装 Transformers
# https://huggingface.co/docs/transformers/installation#install-from-source
!pip install git+https://github.com/huggingface/transformers

Looking in indexes: https://mirrors.ustc.edu.cn/pypi/web/simple
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-sbywrgyj
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-sbywrgyj
  Resolved https://github.com/huggingface/transformers to commit ce4fff0be7f6464d713f7ac3e0bbaafbc6959ae5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.38.0.dev0-py3-none-any.whl size=8505457 sha256=09fe5b6a2d7b582bbffffed29594742aa86413ac637279978743d613e37e2c1c
  Stored in directory: /tmp/pip-ephem-wheel-cache-gmu_xrzw/wheels/04/a3/f1/b88775f8e1665827525b19ac7590250f1038d947067beba9fb
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.2
    Uninstalling transformers-4.36.2:
      Successfully uninstal

In [26]:
# 硬件配置如下： v100s 32G显存  64GB内存  交换内存20G（原本交换内存为0，有CPU内存不足导致进程被kill掉，增加交换内存）
!free -m


              total        used        free      shared  buff/cache   available
Mem:          64307       18103       24509           2       21695       45488
Swap:         20479           0       20479


In [23]:
!nvidia-smi

Sat Feb 17 23:04:41 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:00:0D.0 Off |                    0 |
| N/A   31C    P0    24W / 250W |      4MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
参数如下：
{
    "fp16": {
        "enabled": "auto",
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 16,
        "hysteresis": 2,
        "min_loss_scale": 1
    },

    "bf16": {
        "enabled": "auto"
    },

    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto"
        }
    },

    "scheduler": {
        "type": "WarmupLR",
        "params": {
            "warmup_min_lr": "auto",
            "warmup_max_lr": "auto",
            "warmup_num_steps": "auto"
        }
    },

    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": false
        },

        "overlap_comm": true,
        "contiguous_gradients": true,
        "sub_group_size": 1e8,
        "reduce_bucket_size": "auto",
        "reduce_bucket_size": 134217720, 
        "stage3_prefetch_bucket_size": 52428800, 
        "stage3_max_live_parameters": 1e9,
        "stage3_max_reuse_distance": 1e9,
        "stage3_gather_16bit_weights_on_model_save": true
    },

    "gradient_accumulation_steps": 1, 
    "gradient_clipping": 1.0,
    "steps_per_print": 20,
    "train_batch_size": 1, 
    "train_micro_batch_size_per_gpu": 1, 
    "wall_clock_breakdown": false
}

In [ ]:
# DeepSpeed ZeRO-3 模式单 GPU 训练翻译模型（T5-Large）
! deepspeed --num_gpus=1 translation/run_translation.py \
--deepspeed config/ds_config_zero3.json \
--model_name_or_path t5-3b --per_device_train_batch_size 1 \
--output_dir output_dir --overwrite_output_dir --fp16 \
--do_train --max_train_samples 500 --num_train_epochs 1 \
--dataset_name wmt16 --dataset_config "ro-en" \
--source_lang en --target_lang ro


[2024-02-17 23:17:32,301] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/home/xuwei/jupyterlab/venv/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
[2024-02-17 23:17:34,541] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2024-02-17 23:17:34,541] [INFO] [runner.py:571:main] cmd = /home/xuwei/jupyterlab/venv/bin/python -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None translation/run_translation.py --deepspeed config/ds_config_zero3.json --model_name_or_path t5-3b --per_device_train_batch_size 1 --output_dir output_dir --overwrite_output_dir --fp16 -